In [1]:
import tensorflow as tf

from auramask.losses.perceptual import PerceptualLoss
from auramask.losses.embeddistance import EmbeddingDistanceLoss
from auramask.losses.ssim import SSIMLoss
from auramask.losses.aesthetic import AestheticLoss
from auramask.models.face_embeddings import FaceEmbedEnum

2024-03-11 20:22:58.538791: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 20:22:58.577209: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 20:22:58.577233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 20:22:58.578325: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 20:22:58.584942: I tensorflow/core/platform/cpu_feature_guar

Using TensorFlow backend


/opt/miniconda/envs/unet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Inputs

### Victim Models (F)

In [2]:
F = [FaceEmbedEnum.ARCFACE, FaceEmbedEnum.DEEPID]

### HyperParams

In [3]:
# Backbones are 'vgg', 'alex', 'squeeze'
backbone = 'vgg'

## Loss

### Embeddings Loss
$$
loss \leftarrow \dfrac{1}{\left\|\mathbb{F}\right\|} \sum^{\mathbb{F}}_{f} - \dfrac{f(x) \cdot f(x_{adv})} {\left\| f(x)\right\|_{2}\left\| f(x_{adv})\right\|_{2}}
$$

In [4]:
F_loss = EmbeddingDistanceLoss(F=F)

2024-03-11 20:23:01.090498: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-11 20:23:01.128921: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-11 20:23:01.130349: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

```python
F_set = FaceEmbedEnum.build_F(F)
cossim = CosineSimilarity(axis=-1)
def f_cosine_similarity(x, x_adv, f):
  emb_t = f(x)
  emb_adv = f(x_adv)
  dist = cossim(emb_t, emb_adv)
  dist = tf.negative(dist)
  return dist
```

```python
N = len(F)
def F_loss(x, x_adv):
  loss = 0.0
  for f in F_set:
    loss = tf.add(loss, f_cosine_similarity(x, x_adv, f))
  loss = tf.divide(loss, N)
  return loss
```

### Perceptual Loss ($L_{pips}$)
$$
loss \leftarrow loss + \lambda L_{pips}(x_{adv}, x)
$$

In [5]:
lpips = PerceptualLoss(backbone=backbone, spatial=False)

### Aesthetic Loss ($L_{nima}$)
$$
loss \leftarrow loss + \phi L_{nima}(x_{adv}, x)
$$

In [6]:
laes = AestheticLoss()

### SSIM Loss ($L_{nima}$)
$$
loss \leftarrow loss + \phi L_{ssim}(x_{adv}, x)
$$

In [7]:
ssimloss = SSIMLoss()

### Loss Function

In [8]:
def reface_loss(x, x_adv):
  floss = F_loss(x, x_adv)
  lpipsloss = lpips(x, x_adv)
  return tf.add(floss, lpipsloss)

In [9]:
def auramask_loss(x, x_adv):
  floss = F_loss(x, x_adv)
  lpipsloss = lpips(x, x_adv)
  aesthetic = laes(x, x_adv)
  return tf.add(floss, lpipsloss, aesthetic)

# Test
Testing the giving parts of the loss

In [10]:
import tensorflow_datasets as tfds
from keras_cv.layers import Resizing, Rescaling, Augmenter, RandomContrast, RandomColorDegeneration, RandomColorJitter, RandAugment
from tensorflow.data import AUTOTUNE
BATCH_SIZE = 16

In [11]:
ds, info = tfds.load('lfw',
                     decoders=tfds.decode.PartialDecoding({
                       'image': True,
                     }),
                     with_info=True,
                     download=True,
                     as_supervised=False)

In [12]:
train_ds = ds['train']

In [13]:
SEED = int.from_bytes("Pochita Lives".encode())

augmenter = Augmenter(
  [
    Rescaling(1./255),
    Resizing(256,256),
  ]
)

noise_addition = Augmenter(
  [
    RandAugment(
      value_range=(0,1),
      augmentations_per_image=3,
      magnitude=0.5,
      seed=SEED
      )
  ]
)

def preprocess_data(images, augment=True):
  outputs_1 = augmenter(images)
  if augment:
    outputs_2 = noise_addition(outputs_1)
  else:
    outputs_2 = outputs_1
  return outputs_1, outputs_2

In [14]:
train_ds = train_ds.batch(BATCH_SIZE).map(
  lambda x: preprocess_data(x['image'], True),
  num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

## Total Loss Compute

In [15]:
for batch in train_ds.take(1):
  print(auramask_loss(batch[0], batch[1]))

2024-03-11 20:23:21.765559: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-03-11 20:23:22.501980: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-03-11 20:23:22.563011: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-11 20:23:22.813693: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


tf.Tensor(1.3457596, shape=(), dtype=float32)


## Component Loss

In [16]:
for batch in train_ds.take(5):
  floss = F_loss(batch[0], batch[1])
  ploss = lpips(batch[0], batch[1])
  aloss = laes(batch[0], batch[1])
  ssiml = ssimloss(batch[0], batch[1])
  print("Losses:\n", "\tF-Loss: %0.3f\n"%(floss), "\tLpips: %0.3f\n"%(ploss), "\tAesthetic: %0.3f\n"%(aloss), "\tSSIM: %0.3f\n"%(ssiml))

Losses:
 	F-Loss: 0.869
 	Lpips: 0.533
 	Aesthetic: 0.647
 	SSIM: 0.868

Losses:
 	F-Loss: 0.904
 	Lpips: 0.439
 	Aesthetic: 0.661
 	SSIM: 0.655

Losses:
 	F-Loss: 0.851
 	Lpips: 0.490
 	Aesthetic: 0.643
 	SSIM: 0.752

Losses:
 	F-Loss: 0.888
 	Lpips: 0.447
 	Aesthetic: 0.646
 	SSIM: 0.780



2024-03-11 20:23:29.376005: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Losses:
 	F-Loss: 0.877
 	Lpips: 0.470
 	Aesthetic: 0.633
 	SSIM: 0.752

